In [4]:
from __future__ import division
from IPython.display import display

import time

#from symengine import cos, sin
#from sympy import simplify, transpose, Matrix, symbols, pi, eye
from sympy import *
import sympy as sym
from sympy.interactive import printing
printing.init_printing(use_latex='mathjax')

In [5]:
# MATRICES and additional stuff

def mySimple(expr):
    "how simpify"
    expr = simplify(expand(expr))
    return expr

# !dont remember where need add theta_2 (defaut)
def T(i, j, simp=False):
    "homogeneus matrices, i>j"
    if Ts[i][j] != -1:
        return Ts[i][j]
    else:
        H = eye(4)
        for k in range(i, j):
            Rz = Matrix([[cos(thi[i]), -sin(thi[i]), 0, 0],[sin(thi[i]), cos(thi[i]), 0, 0],[0,0,1,0],[0,0,0,1]])
            Rx = Matrix([[1, 0, 0, 0],[0, cos(alphai[i]), -sin(alphai[i]), 0],[0,sin(alphai[i]), cos(alphai[i]), 0],[0,0,0,1]])
            Tz = Matrix([[1,0,0,0],[0,1,0,0],[0,0,1, di[i]],[0,0,0,1]])
            Tx = Matrix([[1,0,0,ai[i]],[0,1,0,0],[0,0,1, 0],[0,0,0,1]])
            dT = Rz*Tz*Tx*Rx
            H = H * dT
        if simp:
            H = mySimple(H)
        Ts[i][j] = H
        return H
    

def R(i, j, simp=False):
    "rotation matrices, i>j"
    if Rs[i][j] != -1:
        return Rs[i][j]
    else:
        r = T(i,j)[:3, :3]
        if simp:
            r = mySimple(r)
        Rs[i][j] = r
        return r

def getZ(i, simp=True):
    "Z vectors in 0 RF"
    if Zs[i] != -1:
        return Zs[i]
    else:
        z = R(0, i) * Matrix([[0],[0],[1]])
        Zs[i] = z
        return z

# r vectors    
def get_r0_0To(i):
    "r_{0, i}^0 in 0 RF"
    r = T(0, i)[:3, 3]
    return r # 3x1

def get_ri_0To(i, simp=False):
    "r_{0, i} in 0 RF"
    r = transpose(R(0, i)) * get_r0_0To(i)
    if simp:
        r = mySimple(r)
    return r # 3x1

def get_rci(i):
    "rc_{i, ci} in i RF"
    # return Matrix([[xc[i]],[yc[i]],[zc[i]], [1]]) # 4x1
    return Matrix([[xc[i]],[yc[i]],[zc[i]]]) # 3x1
    
def get_rc0(i, simp=False):
    "rci in 0 RF"
    rc0 = T(0, i) * getrci(i)
    if simp:
        rc0 = mySimple(rc0)
    return rc0 # 3x1

def get_g(i, simp=False):
    g = transpose(R(0, i)) * g0
    if simp:
        g = mySimple(g)
    return g # 3x1

def getU(simp=False):
    "potential energy"
    U = Matrix([0])
    for i in range(0, n):
        U = U + m[i] * transpose(get_g(i+1)) * get_ri_0To(i+1) + transpose(get_g(i+1)) * (m[i] * get_rci(i+1))
        print('U for link_' + str(i) + ' was computed!')
    if simp:
        U = mySimple(U)
    return -U

In [67]:
def getJomega(i):
    """
        getJomega(0) = J_{omega1}
    """
    j = Matrix(Jomega[i])
    j = j.reshape(5, 3)
    return transpose(j)

getJomega(1)

⎡0  sin(q₁(t))   0  0  0⎤
⎢                       ⎥
⎢0  -cos(q₁(t))  0  0  0⎥
⎢                       ⎥
⎣1       0       0  0  0⎦

In [72]:
L

NameError: name 'L' is not defined

In [73]:
# INIT VARIABLES AND PARAMETERS FOR LATEX
n = 5
g = 9.82
g0 = Matrix([[0],[0],[-g]])
# q = symbols("q_1 q_2 q_3 q_4 q_5")
# dq = symbols('\dot{q_1} \dot{q_2} \dot{q_3} \dot{q_4} \dot{q_5}')
# ddq = symbols('\ddot{q_1} \ddot{q_2} \ddot{q_3} \ddot{q_4} \ddot{q_5}')
# INIT FUNCTIONS q_i(t)
t = Symbol('t')
q = list()
dq = list()
ddq = list()
for i in range(0, n):
    q.append(Function('q_' + str(i+1))(t))
    dq.append(diff(q[i], t))
    ddq.append(diff(q[i], t, 2))

m = symbols('m_1 m_2 m_3 m_4 m_5')
# vars for radius vectors 0..5
xc = symbols('x_{c0} x_{c1} x_{c2} x_{c3} x_{c4} x_{c5}')
yc = symbols('y_{c0} y_{c1} y_{c2} y_{c3} y_{c4} y_{c5}')
zc = symbols('z_{c0} z_{c1} z_{c2} z_{c3} z_{c4} z_{c5}')
x = symbols('x_{0} x_{1} x_{2} x_{3} x_{4} x_{5}')
y = symbols('y_{0} y_{1} y_{2} y_{3} y_{4} y_{5}')
z = symbols('z_{0} z_{1} z_{2} z_{3} z_{4} z_{5}')
# DH parameters
a = symbols("a_1 a_2 a_3 a_4 a_5")
d = symbols("d_1 d_2 d_3 d_4 d_5")
ai = [a[0], a[1], a[2], 0, 0]
di = [d[0], 0, 0, 0, d[4]]
alphai = [pi/2, 0, 0, pi/2, 0]
thi = [q[0],q[1]+pi/2,q[2],q[3],q[4]]
# init tensors 1..5
Ixx = symbols('I_{1_{xx}} I_{2_{xx}} I_{3_{xx}} I_{4_{xx}} I_{5_{xx}}')
Iyy = symbols('I_{1_{yy}} I_{2_{yy}} I_{3_{yy}} I_{4_{yy}} I_{5_{yy}}')
Izz = symbols('I_{1_{zz}} I_{2_{zz}} I_{3_{zz}} I_{4_{zz}} I_{5_{zz}}')
Ixy = symbols('I_{1_{xy}} I_{2_{xy}} I_{3_{xy}} I_{4_{xy}} I_{5_{xy}}')
Ixz = symbols('I_{1_{xz}} I_{2_{xz}} I_{3_{xz}} I_{4_{xz}} I_{5_{xz}}')
Iyz = symbols('I_{1_{yz}} I_{2_{yz}} I_{3_{yz}} I_{4_{yz}} I_{5_{yz}}')
TENSORs = [-1 for j in range(n+1)]
for i in range(0, n):
    TENSORs[i+1] = Matrix([[Ixx[i], Ixy[i], Ixz[i]],
                         [Ixy[i], Iyy[i], Iyz[i]],
                         [Ixz[i], Iyz[i], Izz[i]]])

"initialization of arrays for frequently use matrices"
Ts = [[-1 for j in range(n+1)] for i in range(n+1)]
Rs = [[-1 for j in range(n+1)] for i in range(n+1)]
Zs = [-1 for j in range(n+1)]

In [48]:
T(0,0)

⎡1  0  0  0⎤
⎢          ⎥
⎢0  1  0  0⎥
⎢          ⎥
⎢0  0  1  0⎥
⎢          ⎥
⎣0  0  0  1⎦

In [75]:
# BLOCK OF VARIABLES FOR CODE GENERATION
xc = symbols('xc0 xc1 xc2 xc3 xc4 xc5')
yc = symbols('yc0 yc1 yc2 yc3 yc4 yc5')
zc = symbols('zc0 zc1 zc2 zc3 zc4 zc5')
x = symbols('x0 x1 x2 x3 x4 x5')
y = symbols('y0 y1 y2 y3 y4 y5')
z = symbols('z0 z1 z2 z3 z4 z6')
# DH parameters
a = symbols("a_1 a_2 a_3 a_4 a_5")
d = symbols("d_1 d_2 d_3 d_4 d_5")
ai = [a[0], a[1], a[2], 0, 0]
di = [d[0], 0, 0, 0, d[4]]
alphai = [pi/2, 0, 0, pi/2, 0]
thi = [q[0],q[1],q[2],q[3],q[4]]
# init tensors 1..5
Ixx = symbols('I1xx I2xx I3xx I4xx I5xx')
Iyy = symbols('I1yy I2yy I3yy I4yy I5yy')
Izz = symbols('I1zz I2zz I3zz I4zz I5zz')
Ixy = symbols('I1xy I2xy I3xy I4xy I5xy')
Ixz = symbols('I1xz I2xz I3xz I4xz I5xz')
Iyz = symbols('I1yz I2yz I3yz I4yz I5yz')

In [77]:
# INIT JACOBIANS
O = Matrix([0,0,0])
Jv = [[O for j in range(n)] for i in range(n)]
Jomega = [[O for j in range(n)] for i in range(n)]

# for v
for i in range(0, n):
    for j in range(0, i+1):
        el = getZ(i).cross(get_r0_0To(j+1) - get_r0_0To(i))
        Jv[i][j] = el

# for omega
for i in range(0, n):
    for j in range(0, i+1):
        el = getZ(j)
        Jomega[i][j] = el
        
def getJv(i):
    "Jacobian liner velocity i=1..5"
    j = Matrix(Jv[i-1])
    j = j.reshape(5,3)
    return transpose(j)

def getJomega(i):
    "Jacobian angular velocity i=1..5"
    j = Matrix(Jomega[i-1])
    j = j.reshape(5,3)
    return transpose(j)

def get_v0(i, simp=False):
    v = getJv(i) * Matrix(dq)
    return v
    
def get_omega0(i, simp=False):
    omega = getJomega(i) * Matrix(dq)
    return omega

def get_vi(i, simp=False):
    v = transpose(R(0, i)) * get_v0(i)
    if simp:
        v = mySimple(v)
    return v # 3x1

def get_omegai(i, simp=False):
    omega = transpose(R(0, i)) * get_omega0(i)
    if simp:
        omega = mySimple(omega)
    return omega # 3x1

In [79]:
# U = getU(True)
# %store U

In [80]:
# LAGRANGIAN
nL = 10
L = [[-1 for i in range(0,nL)] for j in range(0, n)]
tm = time.time
totalT = tm()
print('start computig of L')
for i in range(0, n):
    startTrow = tm()
    l24 = get_vi(i).cross(get_omegai(i)) + get_g(i)
    omegai = get_omegai(i)
    L[i][0] = Rational(1,2) * transpose(get_vi(i)) * get_vi(i) + transpose(get_g(i)) * get_ri_0To(i)
    #L[i][0] = Rational(1,2) * transpose(get_v0(i)) * get_v0(i) + transpose(g0) * get_r0_0To(i)
    L[i][1] = l24[0]
    L[i][2] = l24[1]
    L[i][3] = l24[2]
    L[i][4] = Rational(1,2) * omegai[0]**2
    L[i][5] = Rational(1,2) * omegai[1]**2
    L[i][6] = Rational(1,2) * omegai[2]**2
    L[i][7] = omegai[0]**2 * omegai[1]**2
    L[i][8] = omegai[0]**2 * omegai[2]**2
    L[i][9] = omegai[1]**2 * omegai[2]**2
    for j in range(0, nL):
        startTel = tm()
        # L[i][j] = mySimple(L[i][j])
        print(str(tm() - startTel) + ': j= ' + str(j) + 'is OK!')
    print(str(tm() - startTrow) + ': i= ' + str(i) + ' row of L was computed!')

totalT = tm() - totalT;
print('!total time: ' + str(totalT))

start computig of L
2.384185791015625e-06: j= 0is OK!
1.430511474609375e-06: j= 1is OK!
1.1920928955078125e-06: j= 2is OK!
1.6689300537109375e-06: j= 3is OK!
1.1920928955078125e-06: j= 4is OK!
1.1920928955078125e-06: j= 5is OK!
9.5367431640625e-07: j= 6is OK!
1.1920928955078125e-06: j= 7is OK!
1.1920928955078125e-06: j= 8is OK!
1.1920928955078125e-06: j= 9is OK!
0.4078986644744873: i= 0 row of L was computed!
3.5762786865234375e-06: j= 0is OK!
1.1920928955078125e-06: j= 1is OK!
1.430511474609375e-06: j= 2is OK!
1.1920928955078125e-06: j= 3is OK!
1.430511474609375e-06: j= 4is OK!
1.430511474609375e-06: j= 5is OK!
1.430511474609375e-06: j= 6is OK!
1.6689300537109375e-06: j= 7is OK!
1.430511474609375e-06: j= 8is OK!
1.430511474609375e-06: j= 9is OK!
0.057207584381103516: i= 1 row of L was computed!
2.6226043701171875e-06: j= 0is OK!
1.430511474609375e-06: j= 1is OK!
1.430511474609375e-06: j= 2is OK!
1.1920928955078125e-06: j= 3is OK!
1.9073486328125e-06: j= 4is OK!
1.1920928955078125e-06:

In [81]:
L

⎡⎡                                                                            
⎢⎢⎡⎛⎛  ⎛⎛              2                               ⎞                 2    
⎢⎢⎢⎜⎝- ⎝⎝sin(q₁(t))⋅cos (q₁(t)) - sin(q₁(t))⋅cos(q₁(t))⎠⋅sin(q₁(t)) - sin (q₁(
⎢⎢⎢⎜                                                                          
⎢⎢⎢⎜──────────────────────────────────────────────────────────────────────────
⎣⎣⎣⎝                                                                          

                                                                              
              ⎞ ⎛   ⎛⎛   2             3       ⎞                 2            
t))⋅cos(q₁(t))⎠⋅⎝a₁⋅⎝⎝sin (q₁(t)) + cos (q₁(t))⎠⋅cos(q₁(t)) + sin (q₁(t))⋅cos(
                                                                              
──────────────────────────────────────────────────────────────────────────────
                                                                              

                                                  

In [14]:
def operatorL(L, j, simp=False):
    "!sj and j are swapt, how in report"
    dL_Ddq = diff(L, dq[j])
    dLdq_Dt = diff(dL_Ddq, t)
    dL_Dq = diff(L, q[j])
    opL = dLdq_Dt - dL_Dq
    if simp:
        opL = mySimple(opL)
    return opL

In [27]:
tm = time.time
i = 0
print(tm())
opL_1 = Rational(1,2) * transpose(get_vi(i)) * get_vi(i) + transpose(get_g(i)) * get_ri_0To(i)
print(tm())
opL_1 = operatorL(opL_1, 0)
print(tm())
len(str(opL_1))

1495397249.335128
1495397249.4437997
1495397249.9746053


142557

In [28]:
print(tm())
opL_2 = Rational(1,2) * transpose(get_v0(i)) * get_v0(i) + transpose(g0) * get_r0_0To(i)
print(tm())
opL_2 = operatorL(opL_2, 0)
print(tm())
len(str(opL_2))

1495397253.5554838
1495397253.6072137
1495397254.1646156


142557

In [14]:
thi

[q₁(t), q₂(t), q₃(t), q₄(t), q₅(t)]

In [18]:
CHI = [[0 for j in range(0, nL)] for i in range(0, n)]
for i in range(0, n):
    CHI[i][0] = m[i]
    CHI[i][1] = m[i] * xc[i+1]
    CHI[i][2] = m[i] * yc[i+1]
    CHI[i][3] = m[i] * zc[i+1]
    CHI[i][4] = Ixx[i]
    CHI[i][5] = Iyy[i]
    CHI[i][6] = Izz[i]
    CHI[i][7] = Ixy[i]
    CHI[i][8] = Ixz[i]
    CHI[i][9] = Iyz[i]

In [19]:
TAU = [0 for j in range(0, n+1)]
def get_tau(k, simp=False):
    for j in range(k-1, n):
        for i in range(0, nL):
            opL = operatorL(L[j][i], j)
            try:
                TAU[k] = TAU[k] + CHI[j][i] * opL[0]
            except TypeError:
                TAU[k] = TAU[k] + CHI[j][i] * opL
        print('tau_' + str(j) + ' was computed!')        
    return TAU[k]

In [21]:
import sympy.printing.python as p
opL_sym = operatorL(L[1][0], 0)
opL = sympify(expand(opL_sym))

In [58]:
import sympy
sympy.python(opL[0])

"a_1 = Symbol('a_1')\nt = Symbol('t')\nq_1 = Function('q_1')\ne = a_1**2*sin(q_1(t))**4*Derivative(q_1(t), t, t) + 2*a_1**2*sin(q_1(t))**2*cos(q_1(t))**2*Derivative(q_1(t), t, t) + a_1**2*cos(q_1(t))**4*Derivative(q_1(t), t, t)"

In [98]:
import re

def python_gencode(expr):
   
    def repldq(m):
        "replace Derivativ and replace q_i(t) on dq_i and next dq[i]"
        lala = re.search(r"[d]*q_[1-5]", m.group(0))
        t = re.findall(r"[t]+", m.group(0))
        num = re.search(r"[1-9]", lala.group(0))
        if len(t)-2 == 1:
            return "dq[" + str(num.group(0)) +"]"
        elif len(t)-2 == 2:
            return "ddq[" + str(num.group(0)) + "]"
            
    def rmt(m):
        return ""
    
    def replace_qi(m):
        q_i = re.search(r"q_[1-5]", m.group(0))
        num = re.search(r"[1-9]", q_i.group(0))
        return "q[" + str(num.group(0)) + "]"
    
    # replace all Derivative(.*)
    rerdq = re.sub(r"Derivative\([d]*q_[1-5]+\(t\),[\w ,]+\)", repldq, expr)
    # delete all "(t)"
    remt = re.sub(r"\(t\)", rmt, rerdq)
    # replace all q_i to q[i]
    remt = re.sub(r"q_[1-5]", replace_qi, remt)
    return remt
python_gencode(str(opL[0]))

'a_1**2*sin(q[1])**4*ddq[1] + 2*a_1**2*sin(q[1])**2*cos(q[1])**2*ddq[1] + a_1**2*cos(q[1])**4*ddq[1]'

In [16]:
import sympy.printing.octave as o
import re

def octaveButyfiler(expr):
    ptq = r"[d]*q_[1-5]"
    ptex = r"Derivative\([d]*q_[1-5]+\(t\),[\w ,]+\)"
    ptt = r"\(t\)"
    pt = r"[t]+"

    def repldq(m):
        lala = re.search(ptq, m.group(0))
        t = re.findall(pt, m.group(0))
        if len(t)-2 == 1:
            return "d" + lala.group(0) 
        elif len(t)-2 == 2:
            return "dd" + lala.group(0) 

    def rmt(m):
        return ""

    rerdq = re.sub(ptex, repldq, expr)
    remt = re.sub(ptt, rmt, rerdq)
    return remt

In [ ]:
# f = open('xi', 'w')
# xi = ""
# i = 1
# # xi+NumElementInRow+k
# for k in range(0, nL):
#     print("start_" + str(k+1))
#     xii = simplify(oL1[k])
#     s = o.octave_code(xii, assign_to='xi'+str(i)+'_'+str(k+1), human=True, inline=True)
#     xii = octaveButyfiler(s)
#     xi += str(xii) + '\n'
#     print("end_" + str(k+1) + " from 1 to 10")
# f.write(xi11)

In [23]:
import datetime as dt
tm = time.time
startTmatrix = tm()
print('computing start! ' + str(dt.datetime.fromtimestamp(tm())))
for j in range(0, n):
    startTrow = tm()
    print('\trow_' + str(j) + ' start!')
    # opL = ''
    for i in range(4, n):
        print('\t\telement_' + str(i) + ' start!')
        fileName = 'xi/xi_' + str(j) + str(i)
        fileName_tex = 'xi_tex/xi_' + str(j) + str(i)
        for k in range(7, nL):
            startTelement = tm()
            print('\t\t\tsubElement_' + str(k) + ' start at ' + str(dt.datetime.fromtimestamp(tm())))
            file = open(fileName, 'a') 
            file_tex = open(fileName_tex, 'a') 
            
            opL_sym = operatorL(L[i][k], j)
            print(len(str(opL_sym)))
            opL_sym = simplify(opL_sym)
            print(len(str(opL_sym)))
            
            file_tex.write(str(opL_sym))
            file_tex.close()

            varName = 'opL_' + str(i) + str(k)
            opL_raw = o.octave_code(opL_sym, assign_to=varName, human=True, inline=True)
            opL_oct = octaveButyfiler(opL_raw)
            opL = str(opL_oct) + '\n\n'
            file.write(opL)
            file.close()
            print('\t\t\tsubElement_' + str(k) + ': ' + str(tm() - startTelement) + ' end at ' + str(dt.datetime.fromtimestamp(tm())))
        print('\t\telement_' + str(i) + ': ' + str(tm() - startTrow))
    print('\trow_' + str(j) + ': ' + str(tm() - startTrow))
print('total time on compute matrix \XI: ' + str(tm() - startTmatrix))
print(str(dt.datetime.fromtimestamp(tm())))

computing start! 2017-05-21 21:58:55.203030
	row_0 start!
		element_0 start!
			subElement_7 start at 2017-05-21 21:58:55.205277
20943


KeyboardInterrupt: 

In [ ]:
# compute TAU
# tau5 = get_tau(5)
# s = o.octave_code(tau5, assign_to='tau5', human=True, inline=True)
# tau5 = octaveButyfiler(s)
# tau = get_tau(3)
# s = o.octave_code(tau, assign_to='tau3', human=True, inline=True)
# tau3 = octaveButyfiler(s)

In [135]:
o

'a_1**2*sin(q[1])**4*ddq[1] + 2*a_1**2*sin(q[1])**2*cos(q[1])**2*ddq[1] + a_1**2*cos(q[1])**4*ddq[1]'

In [21]:
opL00 = simplify(opL00)
len(str(opL00))

29753

In [27]:
ooo = len(str(opL00))
ooo

29753

In [ ]:
# # j=1 row of xi
# j=1 #q_1
# # element 1
# i, k = 1, 1
# oL1 = [0 for p in range(0, nL)]
# for k in range(0, nL):
#     oL1[k] = operatorL(L[i-1][k], j-1)

# f = open('xi', 'w')
# xi = ""
# i = 1
# # xi+NumElementInRow+k
# for k in range(0, nL):
#     print("start_" + str(k+1))
#     xii = simplify(oL1[k])
#     s = o.octave_code(xii, assign_to='xi'+str(i)+'_'+str(k+1), human=True, inline=True)
#     xii = octaveButyfiler(s)
#     xi += str(xii) + '\n'
#     print("end_" + str(k+1) + " from 1 to 10")
# f.write(xi11)